# HD30501 simulated differential spectrum

In [ ]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import PyAstronomy.pyasl as pyasl

In [ ]:
# Load phoenix spectrum
pathwave = "/home/jneal/Phd/data/phoenixmodels/WAVE_PHOENIX-ACES-AGSS-COND-2011.fits"
specpath = "/home/jneal/Phd/data/phoenixmodels/HD30501b-lte02500-5.00-0.0.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits"
w_mod = fits.getdata(pathwave)
w_mod /= 10   # turn into nm
flux = fits.getdata(specpath)
hdr = fits.getheader(specpath)
# spec = Spectrum(xaxis=w_mod, flux=flux, header=hdr)


# Cut at 2050 - 2150nm
mask = (w_mod < 2150) & (w_mod > 2050)
w_mod = w_mod[mask]
flux = flux[mask]

# Normalize
flux /= np.median(flux)
maxes = flux.argsort()[-100:][::-1]
flux /= np.median(flux[maxes])
flux[flux<0] = 0

In [ ]:
# Calculated Planet RV 
RV_planet = {"1":2327.66695686, "2a":1152.7794576, "2b":1142.77278133,"3":1101.47665896} #m/s

obs_num = "1"
ref_num = "3"

In [ ]:

nflux_rvobs, wlprime_rvobs = pyasl.dopplerShift(w_mod, flux, RV_planet[obs_num]/1000, edgeHandling=None, fillValue=None)
nflux_rvref, wlprime_rvref = pyasl.dopplerShift(w_mod, flux, RV_planet[ref_num]/1000, edgeHandling=None, fillValue=None)
nflux_rv6, wlprime_rv6 = pyasl.dopplerShift(w_mod, flux, 6+RV_planet[ref_num]/1000, edgeHandling=None, fillValue=None)
nflux_rv12, wlprime_rv12 = pyasl.dopplerShift(w_mod, flux, 12+RV_planet[ref_num]/1000, edgeHandling=None, fillValue=None)
nflux_rvhalf, wlprime_rvhalf = pyasl.dopplerShift(w_mod, flux, 0.5+RV_planet[ref_num]/1000, edgeHandling=None, fillValue=None)


spec_diff = nflux_rvobs - nflux_rvref
spec_diff6 = nflux_rv6 - nflux_rvref
spec_diff12 = nflux_rv12 - nflux_rvref
spec_diffhalf = nflux_rvhalf - nflux_rvref

In [ ]:
print(spec_diff)

In [ ]:
plt.plot(w_mod, nflux_rvobs)
plt.title("PHOENIX-ACES Teff=2500K")
plt.xlabel("Wavelength (nm)")
plt.ylabel("Normalized flux")
plt.xlim([2112,2123])
plt.show()


plt.plot(w_mod, 0.01 * spec_diff)
plt.title("Simulated Spectral Difference")
plt.xlim([2112,2123])
plt.xlabel("Wavelength (nm)")
plt.ylabel("Normalized flux diff")
plt.show()

plt.plot(w_mod, 0.01 * spec_diff)
plt.title("Simulated Spectral Difference")
plt.ylim([-0.02, 0.02])
plt.xlim([2112,2123])
plt.xlabel("Wavelength (nm)")
plt.ylabel("Normalized flux diff")
plt.show()


plt.plot(w_mod, spec_diff6, label="6 km/s")
plt.plot(w_mod, spec_diff, label="1 km/s")
plt.plot(w_mod, spec_diffhalf, label="0.25 km/s")

plt.title("RV Separation")
plt.xlim([2112,2123])
plt.xlabel("Wavelength (nm)")
plt.ylabel("Normalized flux diff")
plt.legend()
plt.show()

In [ ]:
# Amplitude of differnce spectra
rvs = np.arange(-9, 10, 0.2)
avg_amplitude = np.zeros_like(rvs)
max_amplitude = np.zeros_like(rvs)
max_amplitude = []
for i, rv in enumerate(rvs):
    nflux_rv, wlprime_rv = pyasl.dopplerShift(w_mod, flux, rv, edgeHandling=None, fillValue=None)
    diff =  flux - nflux_rv
    new_diff = diff[~np.isnan(diff)]
    # print(abs(diff))
    # print(np.max(new_diff))
    # max_amplitude[i] = np.max(new_diff)
    max_amplitude.append(np.max(abs(new_diff)))
    # print(max_amplitude)
    # avg_amplitude[i] = np.median(new_diff)
    
plt.plot(rvs, max_amplitude)
plt.title("Maximum Differnce")
plt.ylabel("Flux difference")
plt.xlabel("RV difference (km/s)")
plt.show()


plt.plot(rvs, max_amplitude)
plt.title("Maximum Difference")
plt.vlines(x=[-1,1], ymin=-0.5, ymax = 0.7)
plt.ylim(0,0.6)
plt.ylabel("Flux difference")
plt.xlabel("RV difference (km/s)")
plt.show()



# Try a 0 rv shift
nflux_1, wlprime_rv = pyasl.dopplerShift(w_mod, flux, 0, edgeHandling=None, fillValue=None)
diff =  flux - nflux_1
print(diff)